In [1]:
import xml.etree.ElementTree as ET
from __future__ import print_function
from nltk import word_tokenize

In [5]:
def prepare_csv(file_path, output):
    data = ET.parse(file_path)
    reviews = data.getroot()
    # root.tag
    # root.attrib

    dataset = []

    for review in reviews:
        sentences = review[0]
        for sentence in sentences:
            if len(sentence) > 1:
                raw_text = sentence[0].text.strip()
                text = word_tokenize(sentence[0].text.strip())
                raw_text = raw_text.split()
                
                sample = [raw_text, []]
                aspects = []

                opinions = sentence[1]
                for opinion in opinions:
                    start = int(opinion.attrib['from'])
                    end = int(opinion.attrib['to'])
                    if start!=end:
                        aspects.append((start, end))
                
                curr = 0
                ai = 0
                wi = 0
                
                while(wi < len(sample[0])):
                    if ai<len(aspects) and curr == aspects[ai][0]:
                        sample[1].append(1)
                        curr += len(sample[0][wi]) + 1
                        wi += 1
                        while(curr<aspects[ai][1]):
                            sample[1].append(2)
                            curr += len(sample[0][wi]) + 1
                            wi += 1
                        ai += 1
                    else:
                        sample[1].append(0)
                        curr += len(sample[0][wi]) + 1
                        wi += 1
                        
                wi = 0
                while(wi < len(sample[0])):
                    word = sample[0][wi]
                    tokens = word_tokenize(word)
                    if len(tokens)>1:
                        sample[0] = sample[0][0:wi] + tokens + sample[0][wi+1:]
                        if sample[1][wi] == 0:
                            sample[1] = sample[1][0:wi] + [0]*len(tokens) + sample[1][wi+1:]
                        elif sample[1][wi] == 1:
                            sample[1] = sample[1][0:wi] + [1] + [2]*(len(tokens)-1) + sample[1][wi+1:]
                        else:
                            sample[1] = sample[1][0:wi] + [2] * len(tokens) + sample[1][wi+1:]
                    wi += 1
                    
                dataset.append(sample)
    x = open(output+'_x.txt', 'w')
    y = open(output+'_y.txt', 'w')
    
    for i in xrange(len(dataset)-1):
        x.write(u' '.join(dataset[i][0]).encode('utf-8').strip() + '\n')
        y.write(' '.join(map(str, dataset[i][1])) + '\n')
    # to remove the tailing \n
    x.write(' '.join(dataset[-1][0]))
    y.write(' '.join(map(str, dataset[-1][1])))

prepare_csv('Dataset/test.xml', 'Dataset/restaurants_test')
prepare_csv('Dataset/train.xml', 'Dataset/restaurants_train')
 

NameError: name 'xrange' is not defined